# Introduction

**Main Topic**

This notebook is for **Inference using [MMdetection](https://github.com/open-mmlab/mmdetection) without internet** for submition.

Hope to be help to someone who need.

**References**

**Chris Deotte, Grand master** : [How to Install Without Internet](https://www.kaggle.com/c/severstal-steel-defect-detection/discussion/113195)

[**MMdetection Official Documents**](https://mmdetection.readthedocs.io/)


# Install MMdetection from scratch

**Version info.**

- MMdetection 2.6.0
- mmcv-full 1.2.0, torch 1.6, cu102

Because this Competetion is [Notebook Competetion](https://www.kaggle.com/docs/competitions#notebooks-only-FAQ), we need to inference .mp4 video without interent. 

So I made `*.whl` files to install MMdetection, mmcv-full without internet. you can use this files from [mmdetection-v2.6.0 dataset](https://www.kaggle.com/jinssaa/mmdetectionv260).

In [ ]:
! pip install ../input/mmdetectionv260/addict-2.4.0-py3-none-any.whl

In [ ]:
! pip install ../input/mmdetectionv260/mmcv_full-latesttorch1.6.0cu102-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
! pip install ../input/mmdetectionv260/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl

In [ ]:
! pip install ../input/mmdetectionv260/mmdet-2.6.0-py3-none-any.whl

In [ ]:
import copy
import os.path as osp
from glob import glob
from tqdm import tqdm

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv import Config
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import train_detector, set_random_seed, init_detector, inference_detector

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt

import nflimpact

## Model build
In this notebook, I used cascade_rcnn_r50_fpn for baseline. I trained 9epochs and I got a `0.0601` public LB score.
config_file is from [nfl_baseline_cascade_rcnn dataset](https://www.kaggle.com/jinssaa/nflbaselinecascadercnn) which is that I made for this Notebook.

I saved 9epochs pretrained model with configs

In [ ]:
# Specify the path to model config and checkpoint file
config_file = '../input/nflbaselinecascadercnn/cascade_rcnn_r50_fpn.py'
checkpoint_file = '../input/nflbaselinecascadercnn/epoch_9.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [ ]:
test_lists = glob('../input/nfl-impact-detection/test/*.mp4')

In [ ]:
gameKey = []
playID = []
view = []
video_name = []
frame_num = []
left = []
width = []
top = []
height = []


for test_list in tqdm(test_lists):
    file_name = test_list
    video = mmcv.VideoReader(file_name)
    
    for i, frame in enumerate(video):
        result = inference_detector(model, frame)
        for j in range(len(result[0])):
            gameKey.append(int(file_name.split('_')[0].split('/')[-1]))
            playID.append(int(file_name.split('_')[1]))
            view.append(file_name.split('_')[2][:-4])
            video_name.append(file_name.split('/')[-1])
            frame_num.append(i+1)
            left.append(int(result[0][j][0]))
            top.append(int(result[0][j][1]))
            width.append(int(abs(result[0][j][2] - result[0][j][0])))
            height.append(int(abs(result[0][j][3] - result[0][j][1])))

In [ ]:
submission = pd.DataFrame({'gameKey': gameKey,
             'playID': playID,
             'view': view,
             'video': video_name,
             'frame': frame_num,
             'left': left,
             'width': width,
             'top': top,
             'height': height}
             )

In [ ]:
submission.head()

In [ ]:
env = nflimpact.make_env()
env.predict(submission) 